In [ ]:
import pandas as pd
import mlflow
from pycaret.classification import setup, pull, compare_models, plot_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import resample
import pymysql
from sqlalchemy import create_engine
import numpy as np
from scipy.stats import skewnorm
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

In [ ]:
MLFLOW_EXPERIMENT_NAME = 'Investoro_Location'
MLFLOW_TAGS = {'data': 'Investoro_location', 'library': 'pycaret'}

mlflow.set_tracking_uri("http://localhost:5000")

In [ ]:
df_original  = pd.read_csv('data.csv', sep=',')

In [ ]:
df_original .head(10)

In [ ]:
df_original.info()

In [ ]:
df_original [df_original .duplicated()]

In [ ]:
df_original [df_original .duplicated()]

In [ ]:
df_corr_temp = df_original.copy()
if pd.api.types.is_string_dtype(df_corr_temp['BuiltYear']):
    df_corr_temp['BuiltYear_Num'] = pd.to_datetime(df_corr_temp['BuiltYear'], format='%Y', errors='coerce').dt.year
elif pd.api.types.is_datetime64_any_dtype(df_corr_temp['BuiltYear']):
     df_corr_temp['BuiltYear_Num'] = df_corr_temp['BuiltYear'].dt.year
else:
    df_corr_temp['BuiltYear_Num'] = pd.to_numeric(df_corr_temp['BuiltYear'], errors='coerce') # Ostateczna próba

cols_for_corr = ['Area', 'Price', 'BuiltYear_Num', 'Floor', 'Floors','Location', 'CountyNumber', 'CommunityNumber',
                   'RegionNumber','KindNumber']
# Upewnij się, że wszystkie kolumny istnieją i są numeryczne
valid_cols_for_corr = [col for col in cols_for_corr if col in df_corr_temp.columns and pd.api.types.is_numeric_dtype(df_corr_temp[col])]
correlation_matrix = df_corr_temp[valid_cols_for_corr].corr()

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Price')

In [ ]:
df_original .isnull().sum()

In [ ]:
df_cleaned = df_original.copy()
print(f"Rozmiar df_cleaned przed czyszczeniem: {df_cleaned.shape}")

df_cleaned.dropna(subset=['Area', 'Price', 'Location'], inplace=True)
print(f"Rozmiar df_cleaned po usunięciu NaN z Area, Price, Location: {df_cleaned.shape}")
display(df_cleaned.isnull().sum().sort_values(ascending=False).head(15))

In [ ]:
Q1_price = df_cleaned["Price"].quantile(0.25)
Q3_price = df_cleaned["Price"].quantile(0.75)
IQR_price = Q3_price - Q1_price
lower_bound_price = Q1_price - 1.5 * IQR_price
upper_bound_price = Q3_price + 1.5 * IQR_price
df_cleaned = df_cleaned[~((df_cleaned["Price"] < lower_bound_price) | (df_cleaned["Price"] > upper_bound_price))]
print(f"Rozmiar df_cleaned po usunięciu outlierów z Price: {df_cleaned.shape}")

In [ ]:
if "PricePerSquareMeter" in df_cleaned.columns and df_cleaned["PricePerSquareMeter"].isnull().sum() < len(df_cleaned) * 0.8: 
    df_cleaned.dropna(subset=['PricePerSquareMeter'], inplace=True) 
    Q1_ppsm = df_cleaned["PricePerSquareMeter"].quantile(0.25)
    Q3_ppsm = df_cleaned["PricePerSquareMeter"].quantile(0.75)
    IQR_ppsm = Q3_ppsm - Q1_ppsm
    lower_bound_ppsm = Q1_ppsm - 1.5 * IQR_ppsm
    upper_bound_ppsm = Q3_ppsm + 1.5 * IQR_ppsm
    df_cleaned = df_cleaned[~((df_cleaned["PricePerSquareMeter"] < lower_bound_ppsm) | (df_cleaned["PricePerSquareMeter"] > upper_bound_ppsm))]
    print(f"Rozmiar df_cleaned po usunięciu outlierów z PricePerSquareMeter: {df_cleaned.shape}")
else:
    print("Kolumna 'PricePerSquareMeter' nie użyta do usuwania outlierów (brak lub za dużo NaN).")

In [ ]:
Q1_area = df_cleaned["Area"].quantile(0.25)
Q3_area = df_cleaned["Area"].quantile(0.75)
IQR_area = Q3_area - Q1_area
lower_bound_area = Q1_area - 1.5 * IQR_area
upper_bound_area = Q3_area + 1.5 * IQR_area
df_cleaned = df_cleaned[~((df_cleaned["Area"] < lower_bound_area) | (df_cleaned["Area"] > upper_bound_area))]
print(f"Rozmiar df_cleaned po usunięciu outlierów z Area: {df_cleaned.shape}")

In [ ]:
df_cleaned['BuiltYear'] = pd.to_datetime(df_cleaned['BuiltYear'], format='%Y', errors='coerce')
print("Konwersja BuiltYear na datetime w df_cleaned zakończona.")

In [ ]:
print("Informacje o df_cleaned po wszystkich krokach czyszczenia:")
df_cleaned.info()
print("\nBraki danych w df_cleaned (%):")
display(df_cleaned.isnull().sum() / len(df_cleaned) * 100)
print("\nPierwsze wiersze df_cleaned:")
display(df_cleaned.head())

In [ ]:
print(f"Rozmiar df_cleaned przed podziałem na train/holdout: {df_cleaned.shape}")
train_df = df_cleaned.sample(frac=0.9, random_state=42)
holdout_df = df_cleaned.drop(train_df.index)

print(f"Rozmiar zbioru treningowego (train_df): {train_df.shape}")
print(f"Rozmiar zbioru holdout (holdout_df): {holdout_df.shape}")

In [ ]:
def convert_column_types(df_to_convert):
    df_copy = df_to_convert.copy()
    str_cols = ['VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber', 'RegionNumber', 'StreetNumber'] # Dodano StreetNumber
    for col in str_cols:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].astype(str)
    
    # BuiltYear powinno być już datetime, ale upewnijmy się
    if 'BuiltYear' in df_copy.columns and not pd.api.types.is_datetime64_any_dtype(df_copy['BuiltYear']):
         df_copy['BuiltYear'] = pd.to_datetime(df_copy['BuiltYear'], format='%Y', errors='coerce')
    return df_copy

train_df = convert_column_types(train_df)
holdout_df = convert_column_types(holdout_df)

print("\\nTypy danych w train_df po konwersji:")
train_df.info()
print("\\nTypy danych w holdout_df po konwersji:")
holdout_df.info()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Description_vectorizer = TfidfVectorizer(
    max_features=100, 
    stop_words=None,
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.95
)

In [ ]:
print("Przetwarzanie TF-IDF dla zbioru treningowego...")
train_df_copy = train_df.copy() # Pracujemy na kopii
train_df_copy['Description_Clean'] = train_df_copy['Description'].fillna('').astype(str)
train_Description_tfidf_features = Description_vectorizer.fit_transform(train_df_copy['Description_Clean'])

try:
    feature_names = Description_vectorizer.get_feature_names_out()
except AttributeError:
    feature_names = Description_vectorizer.get_feature_names_() 
    
train_Description_tfidf_df = pd.DataFrame(
    train_Description_tfidf_features.toarray(),
    columns=['des_tfidf_' + name for name in feature_names],
    index=train_df_copy.index # Ważne, aby zachować oryginalny indeks
)
print(f"Utworzono {train_Description_tfidf_df.shape[1]} cech TF-IDF dla zbioru treningowego.")

train_df_processed = pd.concat(
    [train_df_copy.drop(columns=['Description', 'Descriptionn_Clean'], errors='ignore'), train_Description_tfidf_df], 
    axis=1
)
# WAŻNE: Usuń wiersze gdzie 'Price' < 20000 (lub inna wartość) dopiero PO przetworzeniu TF-IDF, 
# aby uniknąć problemów z niedopasowaniem indeksów przy konkatenacji.
train_df_processed = train_df_processed[train_df_processed['Price'] >= 20000] 
print(f"Rozmiar train_df_processed po usunięciu cen < 20000: {train_df_processed.shape}")
display(train_df_processed.head())

In [ ]:
print("Przetwarzanie TF-IDF dla zbioru holdout...")
holdout_df_copy = holdout_df.copy() # Pracujemy na kopii
holdout_df_copy['Description_Clean'] = holdout_df_copy['Description'].fillna('').astype(str)
holdout_Description_tfidf_features = Description_vectorizer.transform(holdout_df_copy['Description_Clean']) # Użyj transform

holdout_Description_tfidf_df = pd.DataFrame(
    holdout_Description_tfidf_features.toarray(),
    columns=['des_tfidf_' + name for name in feature_names],
    index=holdout_df_copy.index # Ważne, aby zachować oryginalny indeks
)
print(f"Utworzono {holdout_Description_tfidf_df.shape[1]} cech TF-IDF dla zbioru holdout.")

holdout_df_processed = pd.concat(
    [holdout_df_copy.drop(columns=['Description', 'Description_Clean'], errors='ignore'), holdout_Description_tfidf_df],
    axis=1
)
print(f"Rozmiar holdout_df_processed: {holdout_df_processed.shape}")
display(holdout_df_processed.head())

In [ ]:
categorical_features_initial = [
    'Description','Title', 'BuildingType', 'BuildingCondition', 'TypeOfMarket', 'OwnerType', 'Type', 'OfferFrom',
    'VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber', 'RegionNumber'
]
numeric_features_initial = [
    'Area', 'NumberOfRooms', 'Floor', 'Floors', 'CommunityScore'
]
date_features_initial = ['BuiltYear']

categorical_features_to_use = [col for col in categorical_features_initial if col in train_df_processed.columns]
numeric_features_to_use = [col for col in numeric_features_initial if col in train_df_processed.columns]
# Kolumny des_tfidf_* zostaną dodane do numeric_features wewnątrz setup
date_features_to_use = [col for col in date_features_initial if col in train_df_processed.columns]

ignore_features_list_setup = [ # (...) lista jak w oryginalnym kodzie
    'SaleId', 'OriginalId', 'PortalId', 
    'OfferPrice', 'RealPriceAfterRenovation', 'OriginalPrice',
    'PricePerSquareMeter', 'DateAddedToDatabase', 'DateAdded',
    'DateLastModification', 'DateLastRaises', 'NewestDate',
    'AvailableFrom', 'Link', 'Phone', 'MainImage', 'OtherImages',
    'NumberOfDuplicates', 'NumberOfRaises', 'NumberOfModifications',
    'IsDuplicatePriceLower', 'IsDuplicatePrivateOwner', 'Score', 'ScorePrecision',
    'NumberOfCommunityComments', 'NumberOfCommunityOpinions', 'Archive',
    'SubRegionNumber', 'EncryptedId',
    'StreetNumber' # StreetNumber jest teraz stringiem, jeśli ma za dużo wartości, można go tu dodać
]
ignore_features_final = [col for col in ignore_features_list_setup if col in train_df_processed.columns]

print("--- Informacje przed PyCaret setup ---")
print("Liczba kolumn w train_df_processed:", len(train_df_processed.columns.tolist()))
print("Cechy kategoryczne:", categorical_features_to_use)
print("Cechy numeryczne (początkowe):", numeric_features_to_use)
print("Cechy daty:", date_features_to_use)
print("Ignorowane cechy:", ignore_features_final)
print("------------------------------------")

In [ ]:
import os
# Utwórz dedykowany katalog dla tego testu, jeśli nie istnieje
current_directory = os.getcwd() 
local_mlruns_path = os.path.join(current_directory, "mlruns_DIRECT_LOCAL_TEST") 

if not os.path.exists(local_mlruns_path):
    os.makedirs(local_mlruns_path)
    print(f"Utworzono katalog: {local_mlruns_path}")
else:
    print(f"Katalog już istnieje: {local_mlruns_path}")

absolute_mlruns_path = os.path.abspath(local_mlruns_path)
tracking_uri = f"file:///{absolute_mlruns_path.replace(os.sep, '/')}"
mlflow.set_tracking_uri(tracking_uri)

print(f"MLflow tracking URI ustawione na: {mlflow.get_tracking_uri()}")

# MLFLOW_EXPERIMENT_NAME powinno być zdefiniowane wcześniej w Twoim notebooku
# np. MLFLOW_EXPERIMENT_NAME = 'Investoro_Ceny'

try:
    # Sprawdź, czy eksperyment istnieje
    experiment = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
    if experiment is None:
        experiment_id = mlflow.create_experiment(MLFLOW_EXPERIMENT_NAME)
        print(f"Utworzono nowy eksperyment MLflow: '{MLFLOW_EXPERIMENT_NAME}' o ID: {experiment_id}")
    else:
        experiment_id = experiment.experiment_id
        print(f"Znaleziono istniejący eksperyment: '{MLFLOW_EXPERIMENT_NAME}' o ID: {experiment_id}")
    
    # Ustaw eksperyment jako aktywny
    mlflow.set_experiment(experiment_name=MLFLOW_EXPERIMENT_NAME)
    print(f"Aktywny eksperyment MLflow ustawiony na: '{MLFLOW_EXPERIMENT_NAME}'")

except Exception as e:
    print(f"Błąd podczas ustawiania/tworzenia eksperymentu MLflow: {e}")
    import traceback
    print(traceback.format_exc())


In [ ]:
import pandas as pd
import numpy as np

print(f"Oryginalny kształt danych: {train_df_processed.shape}")

# Funkcja do ekstrakcji dzielnicy z kolumny 'Location'
def extract_district(location_str):
    if not isinstance(location_str, str):
        return np.nan  # Zwróć NaN, jeśli wartość nie jest stringiem
    
    parts = [part.strip() for part in location_str.split(',')]
    
    # === Logika ekstrakcji (można ją dostosować) ===
    # Zakładamy, że interesuje nas miasto i dzielnica, jeśli są dostępne
    # Przykład: 'Mazowieckie, Warszawa, Mokotów' -> 'Warszawa, Mokotów'
    
    if len(parts) >= 3:
        # Bierzemy miasto i dzielnicę
        return f"{parts[1]}, {parts[2]}"
    elif len(parts) == 2:
        # Jeśli jest tylko województwo i miasto, bierzemy miasto
        return parts[1]
    elif len(parts) == 1:
        # Jeśli jest tylko jeden człon, bierzemy go
        return parts[0]
    else:
        return np.nan

# Stwórz nową kolumnę 'District' w swoim głównym DataFrame
# Użyjemy .copy(), aby uniknąć ostrzeżeń SettingWithCopyWarning
train_df_processed = train_df_processed.copy()
train_df_processed['District'] = train_df_processed['Location'].apply(extract_district)

# Usuń wiersze, gdzie nie udało się wyodrębnić dzielnicy
train_df_processed.dropna(subset=['District'], inplace=True)

print(f"Kształt danych po dodaniu kolumny 'District': {train_df_processed.shape}")
print("\nPrzykładowe wyekstrahowane dzielnice:")
print(train_df_processed[['Location', 'District']].head(10))

print(f"\nLiczba unikalnych dzielnic do klasyfikacji: {train_df_processed['District'].nunique()}")

In [ ]:
import pandas as pd
from IPython.display import display

# Ustawiamy próg, ile minimalnie razy musi wystąpić dana lokalizacja
# 2 to absolutne minimum, aby naprawić błąd. Wartości 3-5 mogą dać stabilniejszy model.
MIN_SAMPLES_PER_LOCATION = 3 

# Policz, ile razy występuje każda unikalna lokalizacja
location_counts = train_df_processed['Location'].value_counts()

# Zidentyfikuj lokalizacje, które występują rzadziej niż nasz próg
locations_to_remove = location_counts[location_counts < MIN_SAMPLES_PER_LOCATION].index

# Stwórz nową ramkę danych, usuwając wiersze z rzadkimi lokalizacjami
df_filtered = train_df_processed[~train_df_processed['Location'].isin(locations_to_remove)]

# Wyświetl informację, ile danych zostało usuniętych
print(f"Oryginalna liczba wierszy: {len(train_df_processed)}")
print(f"Liczba usuniętych rzadkich lokalizacji: {len(locations_to_remove)}")
print(f"Liczba wierszy po odfiltrowaniu: {len(df_filtered)}")
print("-" * 30)

In [ ]:
import mlflow

# Sprawdź, czy jest aktywny run i zamknij go, jeśli tak
if mlflow.active_run():
    mlflow.end_run()
    print("Aktywny run został zamknięty.")
else:
    print("Nie znaleziono aktywnego runu do zamknięcia.")

In [ ]:
# --- KROK 2a: FILTROWANIE DANYCH WG NOWEJ KOLUMNY 'District' ---
MIN_SAMPLES_PER_DISTRICT = 3 # Zacznij od 3, w razie błędu zwiększ

district_counts = train_df_processed['District'].value_counts()
districts_to_remove = district_counts[district_counts < MIN_SAMPLES_PER_DISTRICT].index
df_filtered = train_df_processed[~train_df_processed['District'].isin(districts_to_remove)]

print(f"Liczba wierszy po odfiltrowaniu rzadkich dzielnic: {len(df_filtered)}")
print(f"Liczba unikalnych dzielnic po filtrowaniu: {df_filtered['District'].nunique()}")
print("-" * 30)


# --- KROK 2b: SETUP Z NOWYM CELEM ---
class_exp = None
try:
    # Upewnij się, że 'Location' i 'District' są na liście ignorowanych,
    # aby uniknąć wycieku danych.
    features_to_ignore = ignore_features_final + ['Location', 'District']
    
    class_exp = setup(
        data=df_filtered,
        target='District',  # <-- NAJWAŻNIEJSZA ZMIANA
        
        # Reszta parametrów
        log_experiment=False, # Zostawiamy False, aby uniknąć błędów z logowaniem
        experiment_name=MLFLOW_EXPERIMENT_NAME + "_District_Classification",
        categorical_features=categorical_features_to_use,
        numeric_features=numeric_features_to_use + [col for col in df_filtered.columns if 'des_tfidf_' in col],
        date_features=date_features_to_use,
        ignore_features=features_to_ignore, # Używamy zaktualizowanej listy
        session_id=1122,
        html=False,
        verbose=False
    )
except Exception as e:
    print(f"Błąd: {e}")

# ... a następnie komórka z compare_models()

In [ ]:
print("Rozpoczynam porównywanie modeli...")

# Uruchom compare_models i zapisz najlepszy model do zmiennej `best_model`
# Możesz zmienić metrykę sortowania, np. na 'F1'
best_model = compare_models(sort='F1') 

# Po zakończeniu, `pull()` da Ci tabelę z wynikami
print("\n" + "="*50)
print("Tabela z metrykami dla wszystkich modeli:")
all_models_metrics = pull()
display(all_models_metrics)
print("="*50)

# Teraz masz też dostęp do samego obiektu najlepszego modelu
print(f"\nNajlepszy zidentyfikowany model to: {best_model}")
print("\nMożesz teraz używać zmiennej 'best_model' do dalszych operacji (np. tune_model, predict_model).")

In [ ]:
print("Metryki dla najlepszego modelu:")
best_model_metrics = all_models_metrics.head(1)
display(best_model_metrics)

In [ ]:
tuned_model = tune_model(best_model, n_iter=25, sort='F1')

In [ ]:
# Porównaj wyniki przed i po tuningu
print("\n--- Wyniki po tuningu ---")
tuned_metrics = pull()
display(tuned_metrics)

In [ ]:
print("\n--- Najlepszy model przed tuningiem ---")
display(all_models_metrics.head(1))